In [32]:
from pycharmm.psf import delete_atoms
from pycharmm.settings import set_verbosity

from crimm.Adaptors.pyCHARMMAdaptors import load_chain, load_topology, load_parameters
from crimm.Adaptors.pyCHARMMAdaptors import minimize, sync_coords
from crimm.Modeller.LoopBuilder import ChainLoopBuilder
from crimm.Modeller import TopologyLoader, ParameterLoader
from crimm.Modeller.TopoFixer import fix_chain
from crimm.Fetchers import (
    fetch_rcsb, fetch_alphafold, uniprot_id_query, fetch_swiss_model
)

## Fetch a Structure with Missing Loops from RCSB

In [40]:
struct = fetch_rcsb('4BBY')
chain = struct[1]['A']

canonical sequence has all the residue identities for the chain

In [41]:
len(chain.can_seq)

658

In [42]:
chain

NGLWidget()

<Polypeptide(L) id=A Residues=555>
  Description: ALKYLDIHYDROXYACETONEPHOSPHATE SYNTHASE, PEROXISOMAL


The masked sequence `masked_seq` displays the missing residues and their identities

In [43]:
chain.masked_seq.show()

MAEAAAAAAAAAAAGETSASSGSAAERDPDQDRAGRRLRVLSGHLLGRPQEALSTNECKARRAASAATAAPTATPAAPESGIIPKKRQELMKWNGWGYNDSKFFLNKKGQLELTGKRYPLSGVALPTFKDWIQNTFGINLDHKTTSKASLNPSDTPPSIVNEDFLHELKKTNISYSQEADDRVFRAHGHCLHEIFLLREGMFERIPDIVLWPTCHDDVVKIVNLACKYNLCIIPIGGGTSVSYGLMCPADETRTIISLDTSQMNRILWVDENNLTAHVEAGITGQELERQLKESGYCTGHEPDSLEFSTVGGWISTRASGMKKNIYGNIEDLVVHMKVVTPRGVIEKSCQGPRMSTGPDIHHFIMGSEGTLGVITEATIKIRPTPEYQKYGSVAFPNFEQGVACLREIAKQRCAPASIRLMDNQQFQFGHALKPQVSSIFTSFLDGLKKFYITKFKGFDPNQLSVATLLFEGDREKVLQHEKQVYDIAAKFGGLAAGEDNGQRGYLLTYVIAYMRDLGLEYYIIGESFETSAPWDRVVDLCRNVKERIRRECKEKGVQFPPLSTCRVTQTYDAGACIYFYFAFNYRGISDPLAVFEQTEAAAREEILANGGSLSHHHGVGKLRKQWLKESISDVGFGMLKSVKDYVDPTNIFGNRNLL


Alternatively, the `missing_res` attribute also shows the missing residues' sequence number and resname

In [44]:
chain.missing_res

[(1, 'MET'),
 (2, 'ALA'),
 (3, 'GLU'),
 (4, 'ALA'),
 (5, 'ALA'),
 (6, 'ALA'),
 (7, 'ALA'),
 (8, 'ALA'),
 (9, 'ALA'),
 (10, 'ALA'),
 (11, 'ALA'),
 (12, 'ALA'),
 (13, 'ALA'),
 (14, 'ALA'),
 (15, 'GLY'),
 (16, 'GLU'),
 (17, 'THR'),
 (18, 'SER'),
 (19, 'ALA'),
 (20, 'SER'),
 (21, 'SER'),
 (22, 'GLY'),
 (23, 'SER'),
 (24, 'ALA'),
 (25, 'ALA'),
 (26, 'GLU'),
 (27, 'ARG'),
 (28, 'ASP'),
 (29, 'PRO'),
 (30, 'ASP'),
 (31, 'GLN'),
 (32, 'ASP'),
 (33, 'ARG'),
 (34, 'ALA'),
 (35, 'GLY'),
 (36, 'ARG'),
 (37, 'ARG'),
 (38, 'LEU'),
 (39, 'ARG'),
 (40, 'VAL'),
 (41, 'LEU'),
 (42, 'SER'),
 (43, 'GLY'),
 (44, 'HIS'),
 (45, 'LEU'),
 (46, 'LEU'),
 (47, 'GLY'),
 (48, 'ARG'),
 (49, 'PRO'),
 (50, 'GLN'),
 (51, 'GLU'),
 (52, 'ALA'),
 (53, 'LEU'),
 (54, 'SER'),
 (55, 'THR'),
 (56, 'ASN'),
 (57, 'GLU'),
 (58, 'CYS'),
 (59, 'LYS'),
 (60, 'ALA'),
 (61, 'ARG'),
 (62, 'ARG'),
 (63, 'ALA'),
 (64, 'ALA'),
 (65, 'SER'),
 (66, 'ALA'),
 (67, 'ALA'),
 (68, 'THR'),
 (69, 'ALA'),
 (70, 'ALA'),
 (71, 'PRO'),
 (72, 'THR'),
 

`ChainLoopBuilder` is the class that facilitate the loop building using homology models or Alphafold structure. It will query RCSB for sequence search with an identity score cutoff to find the homology models, and if any model contains the loop structure, it will be superimposed and transfer these loop residues to the chain being fixed.

If no homology models available for fixing the chain, AlphaFold structure can also be used.

In [6]:
looper = ChainLoopBuilder(chain)
looper.build_from_homology(
    max_num_match = 10, identity_score_cutoff = 0.95
)

In [7]:
final_chain = looper.get_chain()
final_chain.masked_seq.show()

MAEAAAAAAAAAAAGETSASSGSAAERDPDQDRAGRRLRVLSGHLLGRPQEALSTNECKARRAASAATAAPTATPAAPESGIIPKKRQELMKWNGWGYNDSKFFLNKKGQLELTGKRYPLSGVALPTFKDWIQNTFGINLDHKTTSKASLNPSDTPPSIVNEDFLHELKKTNISYSQEADDRVFRAHGHCLHEIFLLREGMFERIPDIVLWPTCHDDVVKIVNLACKYNLCIIPIGGGTSVSYGLMCPADETRTIISLDTSQMNRILWVDENNLTAHVEAGITGQELERQLKESGYCTGHEPDSLEFSTVGGWISTRASGMKKNIYGNIEDLVVHMKVVTPRGVIEKSCQGPRMSTGPDIHHFIMGSEGTLGVITEATIKIRPTPEYQKYGSVAFPNFEQGVACLREIAKQRCAPASIRLMDNQQFQFGHALKPQVSSIFTSFLDGLKKFYITKFKGFDPNQLSVATLLFEGDREKVLQHEKQVYDIAAKFGGLAAGEDNGQRGYLLTYVIAYMRDLGLEYYIIGESFETSAPWDRVVDLCRNVKERIRRECKEKGVQFPPLSTCRVTQTYDAGACIYFYFAFNYRGISDPLAVFEQTEAAAREEILANGGSLSHHHGVGKLRKQWLKESISDVGFGMLKSVKDYVDPTNIFGNRNLL


In [8]:
looper = ChainLoopBuilder(chain)
# missing terminals will also be built if `include_terminal = True`
looper.build_from_alphafold(include_terminal = False)

In [9]:
final_chain = looper.get_chain()
final_chain.masked_seq.show()

MAEAAAAAAAAAAAGETSASSGSAAERDPDQDRAGRRLRVLSGHLLGRPQEALSTNECKARRAASAATAAPTATPAAPESGIIPKKRQELMKWNGWGYNDSKFFLNKKGQLELTGKRYPLSGVALPTFKDWIQNTFGINLDHKTTSKASLNPSDTPPSIVNEDFLHELKKTNISYSQEADDRVFRAHGHCLHEIFLLREGMFERIPDIVLWPTCHDDVVKIVNLACKYNLCIIPIGGGTSVSYGLMCPADETRTIISLDTSQMNRILWVDENNLTAHVEAGITGQELERQLKESGYCTGHEPDSLEFSTVGGWISTRASGMKKNIYGNIEDLVVHMKVVTPRGVIEKSCQGPRMSTGPDIHHFIMGSEGTLGVITEATIKIRPTPEYQKYGSVAFPNFEQGVACLREIAKQRCAPASIRLMDNQQFQFGHALKPQVSSIFTSFLDGLKKFYITKFKGFDPNQLSVATLLFEGDREKVLQHEKQVYDIAAKFGGLAAGEDNGQRGYLLTYVIAYMRDLGLEYYIIGESFETSAPWDRVVDLCRNVKERIRRECKEKGVQFPPLSTCRVTQTYDAGACIYFYFAFNYRGISDPLAVFEQTEAAAREEILANGGSLSHHHGVGKLRKQWLKESISDVGFGMLKSVKDYVDPTNIFGNRNLL


The `highlight_repaired_gaps()` method will visualize what the repaired loop. However, as you can see in the example below, the loops are not connected to the chain, and structure minimization is needed. For this, we will use pyCHARMM's minimization routine.

In [10]:
looper.highlight_repaired_gaps()

View()

## Generate Topology and Load Parameter for the Chain

Before we load any chain into pyCHARM, the topology and parameter need to be generated

In [19]:
# get residue topology definitions for protein
topo = TopologyLoader('protein')
param = ParameterLoader('protein')
param.fill_ic(topo)

# load these rtf and param files into pyCHARMM too
load_topology(topo)
load_parameters(param)

  
 CHARMM>     read rtf card -
 CHARMM>     name /tmp/tmpisnfjxj3
 VOPEN> Attempting to open::/tmp/tmpisnfjxj3::
 MAINIO> Residue topology file being read from unit  91.
 TITLE> *RTF LOADED FROM CRIMM
 TITLE> 36  2
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  
  
 CHARMM>     read param card -
 CHARMM>     name /tmp/tmpl79xk_jt -
 CHARMM>     flex
 VOPEN> Attempting to open::/tmp/tmpl79xk_jt::

          PARAMETER FILE BEING READ FROM UNIT 91
 TITLE> *PRM LOADED FROM CRIMM
 TITLE> *>>>> CHARMM36 ALL-HYDROGEN PARAMETER FILE FOR PROTEINS <<<<<<<<<<
 TITLE> *>>>>> INCLUDES PHI, PSI CROSS TERM MAP (CMAP) CORRECTION <<<<<<<<
 TITLE> *>>>>>>>>>>>>>>>>>>>>>>>>>> JAN. 2016 <<<<<<<<<<<<<<<<<<<<<<<<<<<<
 TITLE> * ALL COMMENTS TO THE CHARMM WEB SITE: WWW.CHARMM.ORG
 TITLE> *             PARAMETER SET DISCUSSION FORUM
 TITLE> *
 PARMIO> NONBOND, HBOND lists and IMAGE atoms cleared.
 VCLOSE: Closing unit   91 with status "KEEP"
  
 CHARMM>     
  


In [12]:
topo.generate_chain_topology(
    final_chain, first_patch = 'ACE', last_patch = 'CT3', coerce = True
)
param.fill_ic(topo)
param.apply(final_chain.topo_elements)
built_atoms = fix_chain(final_chain)

/home/truman/crimm/crimm/Modeller/ParamLoader.py:167: UserWarning: No cmap found in <Polypeptide(L) id=A Residues=578>.
  warnings.warn(


## Load Chain and Minimize in pyCHARMM

In [13]:
load_chain(final_chain)

  
 CHARMM>     read sequence pdb -
 CHARMM>     name /tmp/tmpq8_ihzlh
 VOPEN> Attempting to open::/tmp/tmpq8_ihzlh::
 MAINIO> Sequence information being read from unit  91.
 TITLE>  *

          RESIDUE SEQUENCE --   578 RESIDUES
          GLY ILE ILE PRO LYS LYS ARG GLN GLU LEU MET LYS TRP ASN GLY TRP GLY TYR ASN ASP 
          SER LYS PHE PHE LEU ASN LYS LYS GLY GLN LEU GLU LEU THR GLY LYS ARG TYR PRO LEU 
          SER GLY VAL ALA LEU PRO THR PHE LYS ASP TRP ILE GLN ASN THR PHE GLY ILE ASN LEU 
          ASP HSD LYS THR THR SER LYS ALA SER LEU ASN PRO SER ASP THR PRO PRO SER ILE VAL 
          ASN GLU ASP PHE LEU HSD GLU LEU LYS LYS THR ASN ILE SER TYR SER GLN GLU ALA ASP 
          ASP ARG VAL PHE ARG ALA HSD GLY HSD CYS LEU HSD GLU ILE PHE LEU LEU ARG GLU GLY 
          MET PHE GLU ARG ILE PRO ASP ILE VAL LEU TRP PRO THR CYS HSD ASP ASP VAL VAL LYS 
          ILE VAL ASN LEU ALA CYS LYS TYR ASN LEU CYS ILE ILE PRO ILE GLY GLY GLY THR SER 
          VAL SER TYR GLY LEU MET CYS PRO

In [14]:
# set verbosity to zero to reduce output from minimization steps
prev_level = set_verbosity(0)
minimize(sd_nstep=300, abnr_nstep=0)
set_verbosity(prev_level)

 CSTRAN: Harmonic Restraints
          ABSOlute type as set number  1.  Number of selected atoms:    578
          Reference coordinates set to main coordinates.
          Mass weighting will NOT be used for new restraints.
          The force constant of       0.00000 will be used.
          An exponent of  2 will be used.
          The XYZ scale factors are:       1.00000       1.00000       1.00000
          A total of      0 atoms are restrained.

 NONBOND OPTION FLAGS: 
     ELEC     VDW      ATOMs    CDIElec  FSHIft   VATOm    VFSWIt  
     BYGRoup  NOEXtnd  NOEWald 
 CUTNB  = 14.000 CTEXNB =999.000 CTONNB = 10.000 CTOFNB = 12.000
 CGONNB =  0.000 CGOFNB = 10.000
 WMIN   =  1.500 WRNMXD =  0.500 E14FAC =  1.000 EPS    =  1.000
 NBXMOD =      5
 There are        0 atom  pairs and        0 atom  exclusions.
 There are        0 group pairs and        0 group exclusions.
 <MAKINB> with mode   5 found  25893 exclusions and  24028 interactions(1-4)
 <MAKGRP> found   8070 group exclusio

Since the coordinates were calculated in pyCHARMM, we need to copy these coord from pyCHARMM to crimm. There is a function provided `sync_coords` for synchronize the structures from both side.

In [15]:
sync_coords(final_chain)

Synchronized: <Polypeptide(L) id=A Residues=578>


We can visualize again to see if the strcuture looks reasonable

In [16]:
looper.highlight_repaired_gaps()

View()

## Working on Multiple Structures

Below is an example for working on multiple strcutures with missing loop regions

In [3]:
test_cases = ['3Q4K','5IEV','4BBY',"6FGQ","7M0C",'6UDL','5VQA','6SSV']
structures = [fetch_rcsb(pdbid, include_solvent = False) for pdbid in test_cases]

/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain O not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:72: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain R not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain S not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain B not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain G not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:72: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(


In [5]:
broken_chains = []
for structure in structures:
    for chain in structure.models[0]:
        if chain.chain_type == 'Polypeptide(L)' and not chain.is_continuous():
            broken_chains.append(chain)
broken_chains

[<Polypeptide(L) id=A Residues=356>,
 <Polypeptide(L) id=B Residues=360>,
 <Polypeptide(L) id=A Residues=284>,
 <Polypeptide(L) id=A Residues=555>,
 <Polypeptide(L) id=B Residues=543>,
 <Polypeptide(L) id=A Residues=229>,
 <Polypeptide(L) id=A Residues=332>,
 <Polypeptide(L) id=A Residues=469>,
 <Polypeptide(L) id=A Residues=368>,
 <Polypeptide(L) id=A Residues=365>]

In [9]:
complete_chains = {}
incomplete_chains = {}
for chain in broken_chains:
    pdbid = chain.parent.parent.id
    print(pdbid, chain)
    looper = ChainLoopBuilder(chain)
    looper.build_from_homology(
        max_num_match = 10, identity_score_cutoff = 0.95,
    )
    if not looper.model_chain.is_continuous():
        print('Building from AlphaFold model')
        looper.build_from_alphafold(include_terminal = False)
    fixed_chain = looper.get_chain()
    if fixed_chain.is_continuous():
        if pdbid not in complete_chains:
            complete_chains[pdbid] = {}
        complete_chains[pdbid][chain.id] = fixed_chain
    else:
        if pdbid not in incomplete_chains:
            incomplete_chains[pdbid] = {}
        incomplete_chains[pdbid][chain.id] = fixed_chain

3Q4K <Polypeptide(L) id=A Residues=356>
3Q4K <Polypeptide(L) id=B Residues=360>
5IEV <Polypeptide(L) id=A Residues=284>


/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain D not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:72: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain D not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:72: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(


4BBY <Polypeptide(L) id=A Residues=555>
4BBY <Polypeptide(L) id=B Residues=543>
6FGQ <Polypeptide(L) id=A Residues=229>


/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain E not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:72: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain E not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:72: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain F not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:72: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:87: UserWarning: Chain E not found in model 1
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/Model.py:72: UserWarning: Insufficient atoms for connect record metalc
  warnings.warn(
/home/truman/crimm/crimm/StructEntities/

7M0C <Polypeptide(L) id=A Residues=332>


...

/home/truman/crimm/crimm/Superimpose/ChainSuperimposer.py:68: UserWarning: Residues are not identical at model chain: GLN-253 and template chain: GLU-253
  warnings.warn(


6SSV <Polypeptide(L) id=A Residues=365>


/home/truman/crimm/crimm/Modeller/LoopBuilder.py:434: UserWarning: No homology model found for 6SSV-A from sequence search!
  warnings.warn(


Building from AlphaFold model


/home/truman/crimm/crimm/Superimpose/ChainSuperimposer.py:68: UserWarning: Residues are not identical at model chain: HIS-108 and template chain: TYR-127
  warnings.warn(


In [10]:
complete_chains

{'3Q4K': {'A': <Polypeptide(L) id=A Residues=366>,
  'B': <Polypeptide(L) id=B Residues=366>},
 '5IEV': {'A': <Polypeptide(L) id=A Residues=298>},
 '4BBY': {'A': <Polypeptide(L) id=A Residues=578>,
  'B': <Polypeptide(L) id=B Residues=578>},
 '6FGQ': {'A': <Polypeptide(L) id=A Residues=245>},
 '7M0C': {'A': <Polypeptide(L) id=A Residues=340>},
 '6UDL': {'A': <Polypeptide(L) id=A Residues=477>},
 '5VQA': {'A': <Polypeptide(L) id=A Residues=412>},
 '6SSV': {'A': <Polypeptide(L) id=A Residues=383>}}

In [20]:
for pdbid, chain_dict in complete_chains.items():
    for chain_id, chain in chain_dict.items():
        # PRO and GLY need special treatment when patched at the N-terminus 
        first_resname = chain.residues[0].resname
        if first_resname == 'PRO':
            first_patch = 'PROP'
        elif first_resname == 'GLY':
            first_patch = 'GLYP'
        else:
            first_patch = 'ACE'
        topo.generate_chain_topology(
            chain, first_patch=first_patch, last_patch='CT3', coerce=True
        )
        param.fill_ic(topo)
        param.apply(chain.topo_elements)
        built_atoms = fix_chain(chain)

/home/truman/crimm/crimm/Modeller/ParamLoader.py:167: UserWarning: No cmap found in <Polypeptide(L) id=A Residues=366>.
  warnings.warn(
/home/truman/crimm/crimm/Modeller/ParamLoader.py:167: UserWarning: No cmap found in <Polypeptide(L) id=B Residues=366>.
  warnings.warn(
/home/truman/crimm/crimm/Modeller/TopoLoader.py:363: UserWarning: Atom NT from (298, 'LEU') is not defined in the topology file!
  warnings.warn(
/home/truman/crimm/crimm/Modeller/TopoLoader.py:363: UserWarning: Atom CAT from (298, 'LEU') is not defined in the topology file!
  warnings.warn(
/home/truman/crimm/crimm/Modeller/TopoLoader.py:363: UserWarning: Atom HNT from (298, 'LEU') is not defined in the topology file!
  warnings.warn(
/home/truman/crimm/crimm/Modeller/TopoLoader.py:363: UserWarning: Atom HT1 from (298, 'LEU') is not defined in the topology file!
  warnings.warn(
/home/truman/crimm/crimm/Modeller/TopoLoader.py:363: UserWarning: Atom HT2 from (298, 'LEU') is not defined in the topology file!
  warning

In [23]:
complete_chains['4BBY']['A']

NGLWidget()

<Polypeptide(L) id=A Residues=578>
  Description: ALKYLDIHYDROXYACETONEPHOSPHATE SYNTHASE, PEROXISOMAL


In [26]:
prev_level = set_verbosity(0)
for pdbid, chain_dict in complete_chains.items():
    for chain_id, chain in chain_dict.items():
        print(pdbid, chain_id)
        load_chain(chain)
        minimize(sd_nstep=300, abnr_nstep=0)
        sync_coords(chain)
        delete_atoms()
        print('---'*20)
set_verbosity(prev_level)

3Q4K A
 ***** Message from SEQRDR ***** THE SYSTEM CONTAINS 74 TITRATABLE GROUPS
 THE USER MUST PREDETERMINE THE PROTONATION STATE THROUGH THE SEQUENCE AND RTF
 HIS -  0  HSD -  7  HSE -  0  HSP -  0  ASP - 19  GLU - 29  LYS - 12  TYR -  7
Adopted Basis Newton-Raphson minimization not performed
Synchronized: <Polypeptide(L) id=A Residues=366>
3Q4K B
 ***** Message from SEQRDR ***** THE SYSTEM CONTAINS 74 TITRATABLE GROUPS
 THE USER MUST PREDETERMINE THE PROTONATION STATE THROUGH THE SEQUENCE AND RTF
 HIS -  0  HSD -  7  HSE -  0  HSP -  0  ASP - 19  GLU - 29  LYS - 12  TYR -  7
Adopted Basis Newton-Raphson minimization not performed
Synchronized: <Polypeptide(L) id=B Residues=366>
5IEV A
 ***** Message from SEQRDR ***** THE SYSTEM CONTAINS 74 TITRATABLE GROUPS
 THE USER MUST PREDETERMINE THE PROTONATION STATE THROUGH THE SEQUENCE AND RTF
 HIS -  0  HSD - 10  HSE -  0  HSP -  0  ASP - 16  GLU - 17  LYS - 21  TYR - 10
Adopted Basis Newton-Raphson minimization not performed
Synchronized: 

0

## Important Notes

This method **does not guarantee** a correct final structure! Always inspect the complete chain visually and identify any artifacts. For example, the loop in Chain A of 5VQA shown below actually is crossed into the alpha helix, and no amount of minimization would fix this issue. You will need to either manually fix it or just use the SwissModel or Alphafold structure directly.

In [29]:
complete_chains['5VQA']['A']

NGLWidget()

<Polypeptide(L) id=A Residues=412>
  Description: Pachytene checkpoint protein 2 homolog


## Fetch Swiss Model or AlphaFold structure for Problematic Chains

In [34]:
entity_id = complete_chains['5VQA']['A'].entity_id
uniprot_id = uniprot_id_query('5VQA', entity_id)

sw_struct = fetch_swiss_model(uniprot_id)

In [36]:
sw_struct

NGLWidget()

<Structure id=Q15645-SwissModel Models=1>
│
├───<Model id=0 Chains=1>
	│
	├───<Polypeptide(L) id=A Residues=412>
	├──────Description: Pachytene checkpoint protein 2 homolog


In [37]:
af_struct = fetch_alphafold(uniprot_id)

In [38]:
af_struct

NGLWidget()

<Structure id=AF-Q15645-F1 Models=1>
│
├───<Model id=1 Chains=1>
	│
	├───<Polypeptide(L) id=A Residues=432>
	├──────Description: Pachytene checkpoint protein 2 homolog
